In [1]:
using LinearAlgebra
using Random

## Генерация данных

In [2]:
function generate_matrix(n::Int64)::Matrix{Float64}
    return rand(-100:0.01:100, n, n)
end
function generate_matrix_delta(n::Int64)::Matrix{Float64}
    return rand(-10:0.01:10, n, n)
end

generate_matrix_delta (generic function with 1 method)

In [3]:
function generate_vector(n::Int64)::Vector{Float64}
    return rand(-100:0.01:100, n)
end
function generate_vector_delta(n::Int64)::Vector{Float64}
    return rand(-10:0.01:10, n)
end

generate_vector_delta (generic function with 1 method)

## Нормы

In [4]:
function euclidean_norm(A)::Float64
    return sqrt(sum(abs2, A))
end

euclidean_norm (generic function with 1 method)

In [5]:
function uniform_norm(x::Vector{Float64})::Float64
    return maximum(abs.(x))
end

function uniform_norm(x::Matrix{Float64})::Float64
    return maximum(sum(abs, x, dims=2))
end

uniform_norm (generic function with 2 methods)

## Число обусловленности матрицы

In [6]:
function get_condition_number(A::Matrix{Float64}, _norm::Function)::Float64
    return _norm(inv(A)) * _norm(A)
end

get_condition_number (generic function with 1 method)

## Коэффициент роста элементов матрицы

In [7]:
function gauss_growth_factor(A::Matrix{Float64})::Float64
    n = size(A, 1)
    max_initial = maximum(abs.(A))
    max_during = max_initial

    A_work = copy(A)

    for k in 1:n-1
        for i in k+1:n
            if A_work[k, k] != 0
                factor = A_work[i, k] / A_work[k, k]
                for j in k:n
                    A_work[i, j] -= factor * A_work[k, j]
                end
            end
        end
        max_during = max(max_during, maximum(abs.(A_work)))
    end
    
    return max_during / max_initial
end

gauss_growth_factor (generic function with 1 method)

## Оценки погрешностей

In [8]:
function error_rounding(
    A::Matrix{Float64},
    p::Int64,
    t::Int64,
    _norm::Function
)::Float64
    nu_A = get_condition_number(A, _norm)
    n = size(A, 1)
    g_A = gauss_growth_factor(A)
    return nu_A * n * g_A / p ^ t
end

error_rounding (generic function with 1 method)

In [9]:
function error_input_data(
    A::Matrix{Float64}, delta_A::Matrix{Float64},
    f::Vector{Float64}, delta_f::Vector{Float64},
    _norm::Function
)::Float64
    nu_A = get_condition_number(A, _norm)
    error_A = _norm(delta_A) / _norm(A)
    error_f = _norm(delta_f) / _norm(f)
    return nu_A * (error_A + error_f)
end

error_input_data (generic function with 1 method)

## Погрешность решения

In [10]:
function error_result(
    A::Matrix{Float64}, delta_A::Matrix{Float64},
    f::Vector{Float64}, delta_f::Vector{Float64},
    _norm::Function
)::Float64
    x = A \ f
    delta_x = (A + delta_A) \ (f + delta_f) - x
    return _norm(delta_x) / _norm(x)
end

error_result (generic function with 1 method)

## Тестирование

In [11]:
A = [
    100. 99.;
    99. 98.
]
delta_A = [
    0. 0.;
    0. 0.
]
f = [
    199.,
    197.
]
delta_f = [
    -.01,
    .01
]

print("Относительная ошибка входных данных: ", error_input_data(A, delta_A, f, delta_f, uniform_norm))
print("\nОтносительная ошибка результата: ", error_result(A, delta_A, f, delta_f, uniform_norm))

Относительная ошибка входных данных: 1.9899999999989821
Относительная ошибка результата: 1.9899999999940312

In [12]:
A = generate_matrix(5)
delta_A = generate_matrix_delta(5)
f = generate_vector(5)
delta_f = generate_vector_delta(5)

_norm = euclidean_norm

print("Относительная ошибка округления: ", error_rounding(A, 2, 11, _norm))
print("\nОтносительная ошибка входных данных: ", error_input_data(A, delta_A, f, delta_f, _norm))
print("\nОтносительная ошибка результата: ", error_result(A, delta_A, f, delta_f, _norm))

Относительная ошибка округления: 0.11516827877965007
Относительная ошибка входных данных: 1.32573555646218
Относительная ошибка результата: 0.1490879688109449